In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model # Importante!
from sklearn.metrics import mean_absolute_error

In [ ]:
#passo 2

# Diretórios dos datasets
train_dir = '/content/boneage-training-dataset/boneage-training-dataset'
train_csv_path = '/content/'  # O arquivo CSV está diretamente em /content/
train_csv_file = os.path.join(train_csv_path, 'boneage-training-dataset.csv')

# Carregar o CSV de treino
train_df = pd.read_csv(train_csv_file)
train_df.rename(columns={'id': 'ImageId', 'boneage': 'BoneAgeInMonths', 'male': 'Male'}, inplace=True)

# Adicionar o caminho completo da imagem ao DataFrame de treino
train_df['ImagePath'] = train_df['ImageId'].apply(lambda x: os.path.join(train_dir, f'{x}.png'))

print("Dados de treino carregados:")
print(train_df.head())
print(train_df.head())

Dados de treino carregados:
   ImageId  BoneAgeInMonths   Male  \
0     1377              180  False   
1     1378               12  False   
2     1379               94  False   
3     1380              120   True   
4     1381               82  False   

                                           ImagePath  
0  /content/boneage-training-dataset/boneage-trai...  
1  /content/boneage-training-dataset/boneage-trai...  
2  /content/boneage-training-dataset/boneage-trai...  
3  /content/boneage-training-dataset/boneage-trai...  
4  /content/boneage-training-dataset/boneage-trai...  
   ImageId  BoneAgeInMonths   Male  \
0     1377              180  False   
1     1378               12  False   
2     1379               94  False   
3     1380              120   True   
4     1381               82  False   

                                           ImagePath  
0  /content/boneage-training-dataset/boneage-trai...  
1  /content/boneage-training-dataset/boneage-trai...  
2  /content/boneage

In [ ]:
#passo 3

# Diretórios dos datasets de teste
test_dir = '/content/boneage-test-dataset/boneage-test-dataset'
test_csv_path = '/content/' # O arquivo CSV está diretamente em /content/
test_csv_file = os.path.join(test_csv_path, 'boneage-test-dataset.csv')

# Carregar o CSV de teste
test_df = pd.read_csv(test_csv_file)
test_df.rename(columns={'Case ID': 'ImageId', 'Sex': 'Male'}, inplace=True)
test_df['Male'] = test_df['Male'].apply(lambda x: 1 if x == 'M' else 0) # Converter sexo para numérico

# Adicionar o caminho completo da imagem ao DataFrame de teste
test_df['ImagePath'] = test_df['ImageId'].apply(lambda x: os.path.join(test_dir, f'{x}.png'))

print("\nDados de teste carregados:")
print(test_df.head())


Dados de teste carregados:
   ImageId  Male                                          ImagePath
0     4360     1  /content/boneage-test-dataset/boneage-test-dat...
1     4361     1  /content/boneage-test-dataset/boneage-test-dat...
2     4362     1  /content/boneage-test-dataset/boneage-test-dat...
3     4363     1  /content/boneage-test-dataset/boneage-test-dat...
4     4364     1  /content/boneage-test-dataset/boneage-test-dat...


In [ ]:
#passo 4

# Separar treino e validação
train_df_split, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Normalização da idade óssea
scaler = MinMaxScaler()
train_df_split['BoneAgeNormalized'] = scaler.fit_transform(train_df_split[['BoneAgeInMonths']])
val_df['BoneAgeNormalized'] = scaler.transform(val_df[['BoneAgeInMonths']])

# Gerador de dados com aumento de dados
image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df_split,
    x_col='ImagePath',
    y_col='BoneAgeNormalized',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='raw'
)

val_gen = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='ImagePath',
    y_col='BoneAgeNormalized',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='raw',
    shuffle=False
)

test_gen = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='ImagePath',
    y_col=None,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='input', # Ou 'raw' se você quiser apenas as imagens
    shuffle=False
)

Found 10088 validated image filenames.
Found 2523 validated image filenames.
Found 200 validated image filenames.


In [ ]:
#passo 5

def create_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        BatchNormalization(),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        BatchNormalization(),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        BatchNormalization(),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1) # Uma saída para a idade óssea normalizada
    ])
    return model

input_shape = (image_size[0], image_size[1], 3)
model = create_cnn_model(input_shape)

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 111, 111, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 54, 54, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 26, 26, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │    44,302,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,397,505 (169.36 MB)

 Trainable params: 44,397,057 (169.36 MB)

 Non-trainable params: 448 (1.75 KB)

None


In [ ]:
#passo 6

# Callbacks
checkpoint_filepath = 'best_boneage_model.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_mae',
    mode='min',
    verbose=1
)

early_stopping_callback = EarlyStopping(
    monitor='val_mae',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Treinamento
epochs = 50 # Você pode ajustar o número de épocas
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=[model_checkpoint_callback, early_stopping_callback]
)

# Plotar o histórico de treinamento
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.legend()
plt.title('MAE')
plt.show()

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 359.7808 - mae: 9.3005
Epoch 1: val_mae improved from inf to 1.24228, saving model to best_boneage_model.h5


316/316 ━━━━━━━━━━━━━━━━━━━━ 1202s 4s/step - loss: 359.0798 - mae: 9.2859 - val_loss: 1.9031 - val_mae: 1.2423
Epoch 2/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 17.5376 - mae: 1.2753
Epoch 2: val_mae did not improve from 1.24228
316/316 ━━━━━━━━━━━━━━━━━━━━ 1242s 4s/step - loss: 17.5407 - mae: 1.2752 - val_loss: 61.3513 - val_mae: 2.9337
Epoch 3/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 2.8276 - mae: 0.5794
Epoch 3: val_mae improved from 1.24228 to 0.37013, saving model to best_boneage_model.h5


316/316 ━━━━━━━━━━━━━━━━━━━━ 1245s 4s/step - loss: 2.8245 - mae: 0.5791 - val_loss: 0.8910 - val_mae: 0.3701
Epoch 4/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 1.7001 - mae: 0.3873
Epoch 4: val_mae improved from 0.37013 to 0.21649, saving model to best_boneage_model.h5


316/316 ━━━━━━━━━━━━━━━━━━━━ 1216s 4s/step - loss: 1.6986 - mae: 0.3872 - val_loss: 0.1656 - val_mae: 0.2165
Epoch 5/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.5079 - mae: 0.2839
Epoch 5: val_mae improved from 0.21649 to 0.18962, saving model to best_boneage_model.h5


316/316 ━━━━━━━━━━━━━━━━━━━━ 1268s 4s/step - loss: 0.5085 - mae: 0.2839 - val_loss: 0.1296 - val_mae: 0.1896
Epoch 6/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.9407 - mae: 0.2834
Epoch 6: val_mae did not improve from 0.18962
316/316 ━━━━━━━━━━━━━━━━━━━━ 1258s 4s/step - loss: 0.9497 - mae: 0.2839 - val_loss: 27.0004 - val_mae: 2.3414
Epoch 7/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 0.7483 - mae: 0.3718
Epoch 7: val_mae did not improve from 0.18962
316/316 ━━━━━━━━━━━━━━━━━━━━ 1210s 4s/step - loss: 0.7472 - mae: 0.3715 - val_loss: 0.0630 - val_mae: 0.2017
Epoch 8/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.2304 - mae: 0.2387
Epoch 8: val_mae did not improve from 0.18962
316/316 ━━━━━━━━━━━━━━━━━━━━ 1287s 4s/step - loss: 0.2305 - mae: 0.2387 - val_loss: 0.1040 - val_mae: 0.2025
Epoch 9/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 0.2258 - mae: 0.2348
Epoch 9: val_mae improved from 0.18962 to 0.15157, saving model to best_boneage_model.h5


316/316 ━━━━━━━━━━━━━━━━━━━━ 1309s 4s/step - loss: 0.2255 - mae: 0.2347 - val_loss: 0.0348 - val_mae: 0.1516
Epoch 10/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.0967 - mae: 0.1920
Epoch 10: val_mae did not improve from 0.15157
316/316 ━━━━━━━━━━━━━━━━━━━━ 1232s 4s/step - loss: 0.0968 - mae: 0.1920 - val_loss: 0.4067 - val_mae: 0.2487
Epoch 11/50
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - loss: 0.0890 - mae: 0.1880
Epoch 11: val_mae did not improve from 0.15157
316/316 ━━━━━━━━━━━━━━━━━━━━ 1266s 4s/step - loss: 0.0890 - mae: 0.1880 - val_loss: 0.0387 - val_mae: 0.1645
Epoch 12/50
 84/316 ━━━━━━━━━━━━━━━━━━━━ 13:36 4s/step - loss: 0.0561 - mae: 0.1745

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

# Carregar o melhor modelo salvo, especificando as funções de perda e métrica
best_model = load_model(
    checkpoint_filepath,
    custom_objects={
        'mse': MeanSquaredError(),
        'mae': MeanAbsoluteError()
    }
)

# Compilar o modelo novamente com as funções especificadas
best_model.compile(optimizer='adam', loss='mse', metrics=['mae']) # Use 'adam' como no treinamento, ou o otimizador que você preferir

print("Modelo carregado com funções de perda e métrica importadas explicitamente.")

Modelo carregado com funções de perda e métrica importadas explicitamente.


In [ ]:
#passo 7

# Avaliar no conjunto de validação
eval_results = best_model.evaluate(val_gen)
print(f'Validação Loss: {eval_results[0]:.4f}')
print(f'Validação MAE (normalizado): {eval_results[1]:.4f}')

# Fazer previsões no conjunto de validação e converter de volta para a escala original
val_preds_normalized = best_model.predict(val_gen)
val_preds = scaler.inverse_transform(val_preds_normalized)

# Obter os valores reais da idade óssea do conjunto de validação
val_true = val_df['BoneAgeInMonths'].values.reshape(-1, 1)[:len(val_preds)]

# Calcular o Mean Absolute Error usando as previsões desnormalizadas e os valores reais
mae = mean_absolute_error(val_true, val_preds)
print(f'Mean Absolute Error no conjunto de validação (idade em meses): {mae:.2f}')

79/79 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - loss: 0.0332 - mae: 0.1464
Validação Loss: 0.0348
Validação MAE (normalizado): 0.1516
79/79 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step
Mean Absolute Error no conjunto de validação (idade em meses): 33.95


In [3]:
#passo 8
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

def predict_bone_age_from_array(model_path, image_array, scaler):
    """
    Carrega o modelo, pré-processa o array da imagem e faz a predição da idade óssea.

    Args:
        model_path (str): Caminho para o arquivo do modelo treinado (ex: 'best_boneage_model.h5').
        image_array (np.array): Array numérico da imagem de raio-x.
        scaler: Objeto MinMaxScaler usado para desnormalizar a idade óssea.

    Returns:
        float: Idade óssea prevista em meses.
    """
    # Carregar o modelo dentro da função
    # É importante especificar as métricas/funções customizadas que foram usadas no treinamento
    model = load_model(
        model_path,
        custom_objects={
            'mse': MeanSquaredError(),
            'mae': MeanAbsoluteError()
        }
    )

    # Pré-processamento da imagem (se necessário, redimensione para 224x224 se o array não vier nesse formato)
    # Assumimos que o image_array já está no formato (altura, largura, canais) e precisa ser normalizado e ter dimensão expandida.
    processed_image_array = image_array / 255.0  # Normalizar pixels para o intervalo [0, 1]
    processed_image_array = np.expand_dims(processed_image_array, axis=0) # Adiciona dimensão de lote (batch)

    normalized_prediction = model.predict(processed_image_array)
    predicted_age = scaler.inverse_transform(normalized_prediction)

    return predicted_age[0][0]

In [2]:
# Supondo que os diretórios 'train' e 'valid' estão no diretório '/content/' do Colab
base_dir = '/content' # Ajuste este caminho se seus dados estiverem em outro lugar

first_val_image_path = os.path.join(base_dir, 'valid', val_df['ImagePath'].iloc[0])
predicted_age = predict_bone_age(first_val_image_path, loaded_model, scaler)
actual_age = val_df['BoneAgeInMonths'].iloc[0]

print(f"Caminho da primeira imagem de validação: {first_val_image_path}")
print(f"Idade óssea prevista: {predicted_age:.2f} meses")
print(f"Idade óssea real: {actual_age} meses")

NameError: name 'os' is not defined